# ODrive Motor Configuration Script

This notebook configures an ODrive motor controller for a specific axis (0 or 1).

**Important:** This will erase existing configuration and reboot the ODrive multiple times.

## 1. Import Required Libraries

In [ ]:
import odrive
import fibre.libfibre
import math
import time

## 2. Select Axis to Configure

Set `axis_id` to either `0` or `1` depending on which axis you want to configure.

In [ ]:
# Configure this value: 0 or 1
axis_id = 0

if axis_id not in [0, 1]:
    raise ValueError("axis_id must be 0 or 1")

print(f"Configuring Axis {axis_id}...")

## 3. Erase Existing Configuration

**Warning:** This will erase all existing configuration on the ODrive!

In [ ]:
# Find ODrive and erase configuration
dev0 = odrive.find_any()

try:
    dev0.erase_configuration()
except fibre.libfibre.ObjectLostError:
    pass

# Reconnect after erase
dev0 = odrive.find_any()
print("Configuration erased and reconnected")

## 4. Set Up Configuration Objects

In [ ]:
# Get axis and its components
ax = dev0.axis0 if axis_id == 0 else dev0.axis1
mo = ax.motor
enc = ax.encoder
contr = ax.controller

print(f"Configured objects for axis {axis_id}")

## 5. Configure Motor and Controller Parameters

In [ ]:
# Motor configuration
mo.config.current_lim = 22.0
mo.config.current_lim_margin = 9.0
mo.config.pole_pairs = 20
mo.config.torque_constant = 0.025  # 8.27/90

# Encoder configuration
enc.config.cpr = 16384

# Controller configuration
contr.config.pos_gain = 60
contr.config.vel_gain = 0.1
contr.config.vel_integrator_gain = 0.2
contr.config.vel_limit = math.inf  # Unlimited (careful!)

print("Motor and controller parameters configured")

## 6. Configure Encoder SPI Settings

In [ ]:
# GPIO and SPI encoder settings
dev0.config.gpio7_mode = 0  # digital
dev0.config.gpio8_mode = 0  # digital
enc.config.abs_spi_cs_gpio_pin = 7 if axis_id == 0 else 8
enc.config.mode = 257  # ABS_SPI

print(f"Encoder SPI configured on GPIO pin {enc.config.abs_spi_cs_gpio_pin}")

## 7. Configure Power and Brake Resistor Settings

In [ ]:
# Power-related configuration
dev0.config.brake_resistance = 2.0
dev0.config.dc_bus_undervoltage_trip_level = 8.0
# dev0.config.dc_bus_overvoltage_trip_level = 56.0
# dev0.config.dc_max_positive_current = 20.0
# dev0.config.dc_max_negative_current = -3.0
dev0.config.enable_brake_resistor = True

print("Power and brake resistor settings configured")

## 8. Save Configuration and Reboot

The ODrive will reboot after saving configuration.

In [ ]:
try:
    dev0.save_configuration()
    dev0.reboot()
except fibre.libfibre.ObjectLostError:
    pass

print("Configuration saved and ODrive rebooted")

## 9. Reconnect After Reboot

In [ ]:
# Reconnect after reboot
dev0 = odrive.find_any()
ax = dev0.axis0 if axis_id == 0 else dev0.axis1
mo = ax.motor
enc = ax.encoder
contr = ax.controller

print("Reconnected to ODrive")

## 10. Calibrate Motor

This runs the full motor calibration sequence.

In [ ]:
print("Starting motor calibration...")
ax.requested_state = 4  # FULL_CALIBRATION_SEQUENCE

while not mo.is_calibrated:
    time.sleep(2)
    print(f"motor {axis_id} not calibrated")
    print(mo.error)

mo.config.pre_calibrated = True
print("Motor calibration complete!")

## 11. Calibrate Encoder

This runs the encoder offset calibration.

In [ ]:
print("Starting encoder calibration...")
ax.requested_state = 7  # ENCODER_OFFSET_CALIBRATION

while not enc.is_ready:
    time.sleep(2)
    print(f"encoder {axis_id} not calibrated")
    print(enc.error)

enc.config.pre_calibrated = True
print("Encoder calibration complete!")

## 12. Enable Startup Closed Loop and Save Final Configuration

In [ ]:
# Enable startup closed loop control for this axis
ax.config.startup_closed_loop_control = True

# Clear any errors
dev0.clear_errors()

# Save final configuration
try:
    dev0.save_configuration()
except fibre.libfibre.ObjectLostError:
    pass

print(f"Axis {axis_id} successfully configured and calibrated.")

## Configuration Complete!

Your ODrive axis is now configured and calibrated. The motor will automatically enter closed-loop control on startup.